In [1]:
import pandas as pd

from nltk.tokenize import word_tokenize
from string import punctuation
import nltk
from unicodedata import normalize
from nltk.stem import RSLPStemmer
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')

import pyterrier as pt

import time

pt.init()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [2]:
dados = pd.read_csv("../base_20230428_douglas.csv")

In [3]:
colId = range(len(dados))

In [4]:
dados

,codProposicao,txtSiglaTipo,numAno,numNumero,txtNome,txtEmenta,txtExplicacaoEmenta,txtIndexacao,txtInteiroTeor
0,16357,PL,1999,1165,PL 1165/1999,"Altera dispositivo da Lei nº 8.987, de 13 de f...",Estabelece que as concessionárias disponibiliz...,"Alteração, Lei das Concessões de Serviços Públ...",Ofício nº 1416 (SF) ...
1,19098,PL,1992,3097,PL 3097/1992,Dispõe sobre a eleição de diretores de fundos ...,NaN,"NORMAS, ELEIÇÃO DIRETA, EMPREGADO, APOSENTADO,...",COMISSÃO DE CONSTITUIÇÃO E JUSTIÇA E DE REDAÇÃ...
2,20464,PL,2000,3927,PL 3927/2000,Altera a composição dos Tribunais Regionais do...,"Altera a composição do TRT da 5ª região, 6ª re...","Alteração, Lei Federal, composição, Tribunal R...","COMISSÃO DE TRABALHO, DE ADMINISTRAÇÃO E SERVI..."
3,20683,PL,1998,4117,PL 4117/1998,Dispõe sobre o acesso a ambientes de uso colet...,NaN,"Autorização, pessoa portadora de deficiência, ...",Câmara dos Deputados\n Departamento ...
4,20857,PL,1998,4395,PL 4395/1998,Estabelece as Diretrizes Nacionais de Defesa C...,NaN,"Fixação, diretrizes, defesa civil, (Sindec), d...",- 1 - \nCOMISSÃO DE CONSTITUIÇÃO E JUSTIÇA E D...
...,...,...,...,...,...,...,...,...,...
57104,2358873,PL,2023,2233,PL 2233/2023,Altera o parágrafo único do art. 71 do Código ...,NaN,NaN,1 \n \n \n \n \nCÂMARA DOS DEPUTADOS \nPROJET...
57105,2358874,PL,2019,3616,PL 3616/2019,"Altera a Lei nº 9.503, de 23 de setembro de 19...",NaN,NaN,"Altera a Lei nº 9.503, de 23 de setembro\nde 1..."
57106,2358875,PL,2019,1822,PL 1822/2019,"Altera a Lei nº 11.340, de 7 de agosto de 2006...",NaN,NaN,"Altera a Lei nº 11.340, de 7 de agosto de\n200..."
57107,2358877,PL,2019,3815,PL 3815/2019,"Altera a Lei nº 7.565, de 19 de dezembro de 19...",NaN,NaN,"Altera a Lei nº 7.565, de 19 de dezembro\nde ..."


In [5]:
dados = dados[["txtNome","txtIndexacao"]]
dados1 = dados.assign(id=colId)

In [6]:
dados1 = dados1.fillna("em branco")

In [7]:
dados1 = dados1.rename(columns={'txtIndexacao': 'text','id':'docno'})

In [8]:
dados1

,txtNome,text,docno
0,PL 1165/1999,"Alteração, Lei das Concessões de Serviços Públ...",0
1,PL 3097/1992,"NORMAS, ELEIÇÃO DIRETA, EMPREGADO, APOSENTADO,...",1
2,PL 3927/2000,"Alteração, Lei Federal, composição, Tribunal R...",2
3,PL 4117/1998,"Autorização, pessoa portadora de deficiência, ...",3
4,PL 4395/1998,"Fixação, diretrizes, defesa civil, (Sindec), d...",4
...,...,...,...
57104,PL 2233/2023,em branco,57104
57105,PL 3616/2019,em branco,57105
57106,PL 1822/2019,em branco,57106
57107,PL 3815/2019,em branco,57107


# Config 1

In [10]:
a = time.time()

In [11]:
iter_indexer = pt.IterDictIndexer("E:\Corpora_em_json\experimentos_pyterrier\config1t",stemmer=None,stopwords=None,tokeniser="UTFTokeniser")
indexref2 = iter_indexer.index(dados1.to_dict(orient="records"))

In [12]:
d = time.time()

In [13]:
print("Duração: %f" %(d-a))

Duração: 17.336833


# Config 5

In [14]:
dados = pd.read_csv("../base_20230428_douglas.csv")

In [15]:
colId = range(len(dados))

In [16]:
dados = dados[["txtNome","txtIndexacao"]]
dados1 = dados.assign(id=colId)

In [17]:
dados1 = dados1.fillna("em branco")

In [18]:
dados1 = dados1.rename(columns={'txtIndexacao': 'text','id':'docno'})

In [19]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    terms = " ".join(terms)
    return terms

In [20]:
dados1["text"] = dados1["text"].map(preprocess)

In [21]:
dados1

,txtNome,text,docno
0,PL 1165/1999,alteracao lei concessoes servicos publicos obr...,0
1,PL 3097/1992,normas eleicao direta empregado aposentado dir...,1
2,PL 3927/2000,alteracao lei federal composicao tribunal regi...,2
3,PL 4117/1998,autorizacao pessoa portadora deficiencia defic...,3
4,PL 4395/1998,fixacao diretrizes defesa civil sindec definic...,4
...,...,...,...
57104,PL 2233/2023,branco,57104
57105,PL 3616/2019,branco,57105
57106,PL 1822/2019,branco,57106
57107,PL 3815/2019,branco,57107


In [22]:
a = time.time()

In [23]:
iter_indexer = pt.IterDictIndexer("E:\Corpora_em_json\experimentos_pyterrier\config5t",stemmer=None,stopwords=None,tokeniser="UTFTokeniser")
indexref2 = iter_indexer.index(dados1.to_dict(orient="records"))

In [24]:
d = time.time()

In [25]:
print("Duração: %f" %(d-a))

Duração: 14.515021


# Config 8

In [26]:
dados = pd.read_csv("../base_20230428_douglas.csv")

In [27]:
colId = range(len(dados))

In [28]:
dados = dados[["txtNome","txtIndexacao"]]
dados1 = dados.assign(id=colId)

In [29]:
dados1 = dados1.fillna("em branco")

In [30]:
dados1 = dados1.rename(columns={'txtIndexacao': 'text','id':'docno'})

In [31]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [32]:
dados1["text"] = dados1["text"].map(preprocess)

In [33]:
a = time.time()

In [34]:
iter_indexer = pt.IterDictIndexer("E:\Corpora_em_json\experimentos_pyterrier\config8t",stemmer=None,stopwords=None,tokeniser="UTFTokeniser")
indexref2 = iter_indexer.index(dados1.to_dict(orient="records"))

In [35]:
d = time.time()

In [36]:
print("Duração: %f" %(d-a))

Duração: 14.213581


# Config 9

In [27]:
class Savoy:

    def __removeAllPTAccent(self, old_word):
        word = list(old_word)
        len_word = len(word)-1
        for i in range(len_word, -1, -1):
            if word[i] == 'ä':
                word[i] = 'a'
            if word[i] == 'â':
                word[i] = 'a'
            if word[i] == 'à':
                word[i] = 'a'
            if word[i] == 'á':
                word[i] = 'a'
            if word[i] == 'ã':
                word[i] = 'a'
            if word[i] == 'ê':
                word[i] = 'e'
            if word[i] == 'é':
                word[i] = 'e'
            if word[i] == 'è':
                word[i] = 'e'
            if word[i] == 'ë':
                word[i] = 'e'
            if word[i] == 'ï':
                word[i] = 'i'
            if word[i] == 'î':
                word[i] = 'i'
            if word[i] == 'ì':
                word[i] = 'i'
            if word[i] == 'í':
                word[i] = 'i'
            if word[i] == 'ü':
                word[i] = 'u'
            if word[i] == 'ú':
                word[i] = 'u'
            if word[i] == 'ù':
                word[i] = 'u'
            if word[i] == 'û':
                word[i] = 'u'
            if word[i] == 'ô':
                word[i] = 'o'
            if word[i] == 'ö':
                word[i] = 'o'
            if word[i] == 'ó':
                word[i] = 'o'
            if word[i] == 'ò':
                word[i] = 'o'
            if word[i] == 'õ':
                word[i] = 'o'
            if word[i] == 'ç':
                word[i] = 'c'

        new_word = "".join(word)
        return new_word

    def __finalVowelPortuguese(self, word):
        len_word = len(word)
        if len_word > 3:
            if word[-1] == 'e' or word[-1] == 'a' or word[-1] == 'o':
                word = word[:-1]

        return word

    def __remove_PTsuffix(self, word):
        len_word = len(word)

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'e' and (word[-3] == 'r' or word[-3] == 's' or word[-3] == 'z' or word[-3] == 'l'):
                word = word[:-2]
                return word
        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'n':
                new_word = list(word)
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if (word[-1] == 's' and word[-2] == 'i') and (word[-3] == 'e' or word[-3] == 'é'):
                new_word = list(word)
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'a':
                new_word = list(word)
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'ó':
                new_word = list(word)
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i':
                new_word = list(word)
                new_word[-1] = 'l'
                sing = "".join(new_word)
                return sing

        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'õ':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'ã':
                new_word = list(word)
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 5:
            if word[-1] == 'e' and word[-2] == 't' and word[-3] == 'n' and word[-4] == 'e' and word[-5] == 'm':
                word = word[:-5]
                return word

        if len_word > 2:
            if word[-1] == 's':
                word = word[:-1]

        return word

    def __normFemininPortuguese(self, word):

        len_word = len(word)

        if len_word < 3 or word[-1] != 'a':
            return word

        if len_word > 6:

            if word[-2] == 'h' and word[-3] == 'n' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'c' and word[-3] == 'a' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'r' and word[-3] == 'i' and word[-4] == 'e':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        if len_word > 5:
            if word[-2] == 'n' and word[-3] == 'o':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'r' and word[-3] == 'o':
                word = word[:-1]
                return word

            if word[-2] == 's' and word[-3] == 'o':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 's' and word[-3] == 'e':
                new_word = list(word)
                new_word[-3] = 'ê'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'c' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'v' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'm' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'n':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        return word

    def stem(self, word):
        len_word = len(word)
        if len_word > 2:
            word = self.__remove_PTsuffix(word)
            word = self.__normFemininPortuguese(word)
            word = self.__finalVowelPortuguese(word)
            word = self.__removeAllPTAccent(word)

        return word

In [28]:
dados = pd.read_csv("../base_20230428_douglas.csv")

In [29]:
colId = range(len(dados))

In [30]:
dados = dados[["txtNome","txtIndexacao"]]
dados1 = dados.assign(id=colId)

In [31]:
dados1 = dados1.fillna("em branco")

In [32]:
dados1 = dados1.rename(columns={'txtIndexacao': 'text','id':'docno'})

In [33]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [34]:
dados1["text"] = dados1["text"].map(preprocess)

In [35]:
iter_indexer = pt.IterDictIndexer("E:\Corpora_em_json\experimentos_pyterrier\config9t",stemmer=None,stopwords=None,tokeniser="UTFTokeniser")
indexref2 = iter_indexer.index(dados1.to_dict(orient="records"))

# Config 18

In [37]:
dados = pd.read_csv("../base_20230428_douglas.csv")

In [38]:
colId = range(len(dados))

In [39]:
dados = dados[["txtNome","txtIndexacao"]]
dados1 = dados.assign(id=colId)

In [40]:
dados1 = dados1.fillna("em branco")

In [41]:
dados1 = dados1.rename(columns={'txtIndexacao': 'text','id':'docno'})

In [42]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)

In [43]:
dados1["text"] = dados1["text"].map(preprocess)

In [44]:
a = time.time()

In [45]:
iter_indexer = pt.IterDictIndexer("E:\Corpora_em_json\experimentos_pyterrier\config18t",stemmer=None,stopwords=None,tokeniser="UTFTokeniser")
indexref2 = iter_indexer.index(dados1.to_dict(orient="records"))

In [46]:
d = time.time()

In [47]:
print("Duração: %f" %(d-a))

Duração: 17.367999


# Config 21

In [44]:
dados = pd.read_csv("../base_20230428_douglas.csv")

In [45]:
colId = range(len(dados))

In [46]:
dados = dados[["txtNome","txtIndexacao"]]
dados1 = dados.assign(id=colId)

In [47]:
dados1 = dados1.fillna("em branco")

In [48]:
dados1 = dados1.rename(columns={'txtIndexacao': 'text','id':'docno'})

In [49]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)

In [50]:
dados1["text"] = dados1["text"].map(preprocess)

In [51]:
iter_indexer = pt.IterDictIndexer("E:\Corpora_em_json\experimentos_pyterrier\config21t",stemmer=None,stopwords=None,tokeniser="UTFTokeniser")
indexref2 = iter_indexer.index(dados1.to_dict(orient="records"))

# Config 22

In [52]:
class RSLP_S:
    def __plural_reduction(self, word):
        excep = ["lápis","cais","mais","crúcis","biquínis","pois","depois","dois","leis" ]
        excep_s = ["aliás","pires","lápis","cais","mais","mas","menos", "férias","fezes","pêsames","crúcis","gás", "atrás","moisés","através","convés","ês", "país","após","ambas","ambos","messias"]

        len_word = len(word)
        new_word = list(word)

        if len_word >= 3:
            if new_word[-1] == 's' and new_word[-2] == 'n':
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'õ':
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return  sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'ã':
                if word == 'mães':
                    word = word[:-1]
                    return word
                else:
                    new_word[-2] = 'o'
                    sing = "".join(new_word)
                    sing = sing[:-1]
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'a':
                if word != 'cais' and word != 'mais':
                    new_word[-2] = 'l'
                    sing = "".join(new_word)
                    sing = sing[:-1]
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'é':
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'e':
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'ó':
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i':
                if word not in excep:
                    new_word[-1] = 'l'
                    sing = "".join(new_word)
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'l':
                word = word[:-2]
                return word

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'r':
                word = word[:-2]
                return word

            if new_word[-1] == 's':
                if word not in excep_s:
                    word = word[:-1]

        return word

    def stem(self, word):
        word = self.__plural_reduction(word)

        return word

In [53]:
dados = pd.read_csv("../base_20230428_douglas.csv")

In [54]:
colId = range(len(dados))

In [55]:
dados = dados[["txtNome","txtIndexacao"]]
dados1 = dados.assign(id=colId)

In [56]:
dados1 = dados1.fillna("em branco")

In [57]:
dados1 = dados1.rename(columns={'txtIndexacao': 'text','id':'docno'})

In [58]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    return " ".join(terms)

In [59]:
dados1["text"] = dados1["text"].map(preprocess)

In [60]:
iter_indexer = pt.IterDictIndexer("E:\Corpora_em_json\experimentos_pyterrier\config22t",stemmer=None,stopwords=None,tokeniser="UTFTokeniser")
indexref2 = iter_indexer.index(dados1.to_dict(orient="records"))

# Config 23

In [61]:
dados = pd.read_csv("../base_20230428_douglas.csv")

In [62]:
colId = range(len(dados))

In [63]:
dados = dados[["txtNome","txtIndexacao"]]
dados1 = dados.assign(id=colId)

In [64]:
dados1 = dados1.fillna("em branco")

In [65]:
dados1 = dados1.rename(columns={'txtIndexacao': 'text','id':'docno'})

In [66]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)

In [67]:
dados1["text"] = dados1["text"].map(preprocess)

In [68]:
iter_indexer = pt.IterDictIndexer("E:\Corpora_em_json\experimentos_pyterrier\config23t",stemmer=None,stopwords=None,tokeniser="UTFTokeniser")
indexref2 = iter_indexer.index(dados1.to_dict(orient="records"))